In [1]:
import os
from datetime import datetime
import numpy as np
import h5py
import re

In [2]:
# where are located old precomputed ratio files
dir = r'D:\radary_cartesian'
# output directory (should be the same as outdir in config file)
outdir = r'C:\Users\marti\kinit\radar-dataset-preprocess\output'
rootdir = r'C:\Users\marti\kinit\radar-dataset-preprocess\data'

In [3]:
# get paths to all hdf files in dir and its subdirectories
filepaths = []
for dirname, dirs, files in os.walk(dir):
    for filename in files:
        filename_without_extension, extension = os.path.splitext(filename)
        if extension == '.h5':
            filepaths.append(os.path.join(dirname, filename))

In [4]:
print(filepaths[4])

D:\radary_cartesian\20160105.h5


In [9]:
# load ratios file from dir
ratios = np.array([], dtype=np.float64)
timestamps = []
i = 0
for file in filepaths:
    hf = h5py.File(file, 'r')
    # save them to variables
    ratios = np.append(ratios, hf['ratios'])
    for item in hf['timestamps']:
        timestamps.append(item.decode())
    hf.close()
    i += 1
    if i % 100 == 0:
        print(f'Done file number {i}/{len(filepaths)}.')

Done file number 100/1245.
Done file number 200/1245.
Done file number 300/1245.
Done file number 400/1245.
Done file number 500/1245.
Done file number 600/1245.
Done file number 700/1245.
Done file number 800/1245.
Done file number 900/1245.
Done file number 1000/1245.
Done file number 1100/1245.
Done file number 1200/1245.


In [10]:
ratios.shape[0]

355761

In [11]:
len(timestamps)

355761

In [12]:
ratios[2]

0.0025

In [13]:
timestamps[2058]

'2016-01-10T04:35:00.000000Z'

In [14]:
if ratios.shape[0] != len(timestamps):
    print('Different size of ratios and timestamps arrays. Proceed with caution.')
else:
    print('All good.')

All good.


In [16]:
hf = h5py.File(os.path.join(outdir, 'ratios.h5'), 'a')
hf.create_dataset('ratios', data=ratios, chunks=True)
hf.create_dataset('timestamps', data=timestamps, chunks=True)
hf.close()